In [3]:
from pathlib import Path
import numpy as np

In [13]:
n_ensemble = 1
seeds = list(range(n_ensemble))
lead_times = [1, 2, 3, 5, 7, 10]
holdout_fractions = np.round(np.linspace(0, 1, 11)[:-1] * 10) / 10

In [4]:
master_config = "/home/gsnearing/projects/data_assimilation/master_configs/simulation_config.yml"
for seed in seeds:
    with open(master_config, 'r') as f:
        filedata = f.read()
    filedata = filedata.replace("seed:", f"seed: {seed}")
    filedata = filedata.replace("experiment_name:", f"experiment_name: simulation_{seed}")
    new_config = f"/home/gsnearing/projects/data_assimilation/run_configs/simulation/simulation_{seed}.yml"
    with open(new_config, 'w') as f:
        f.write(filedata)

In [8]:
master_config = "/home/gsnearing/projects/data_assimilation/master_configs/autoregression_holdout_config.yml"
for seed in seeds:
    for lead in lead_times:
        for holdout in holdout_fractions:
            if lead > 1: continue
            with open(master_config, 'r') as f:
                filedata = f.read()
            filedata = filedata.replace("seed:", f"seed: {seed}")
            filedata = filedata.replace("experiment_name:", f"experiment_name: autoregression_{lead}_{holdout}_{seed}")
            filedata = filedata.replace(f"- xxx1\n", f"- {lead}\n")
            filedata = filedata.replace(f"- xxxQObs(mm/d)_shift1\n", f"- QObs(mm/d)_shift{lead}\n")
            filedata = filedata + f'random_holdout: \n '
            filedata = filedata + f'  QObs(mm/d)_shift{lead}: {holdout} \n '
            new_config = f"/home/gsnearing/projects/data_assimilation/run_configs/autoregression/autoregression_{lead}_{holdout}_{seed}.yml"
            with open(new_config, 'w') as f:
                f.write(filedata)

In [4]:
# master_config = "/home/gsnearing/projects/data_assimilation/master_configs/autoregression_noholdout_config.yml"
# for seed in range(n_seeds):
#     for lead in lead_times:
#         with open(master_config, 'r') as f:
#             filedata = f.read()
#         filedata = filedata.replace("seed:", f"seed: {seed}")
#         filedata = filedata.replace("experiment_name:", f"experiment_name: autoregression_{lead}_{seed}")
#         filedata = filedata.replace(f"- xxx1\n", f"- {lead}\n")
#         filedata = filedata.replace(f"- xxxQObs(mm/d)_shift1\n", f"- QObs(mm/d)_shift{lead}\n")
#         filedata = filedata + f'random_holdout: \n '
#         filedata = filedata + f'  QObs(mm/d)_shift{lead}: 0 \n '
#         new_config = f"/home/gsnearing/projects/data_assimilation/run_configs/autoregression_noholdout/autoregression_config_{lead}_{seed}.yml"
#         with open(new_config, 'w') as f:
#             f.write(filedata)

In [18]:
from sklearn.model_selection import KFold

n_kfold = 10

basin_file = Path("/home/gsnearing/projects/data_assimilation/531_basin_list.txt")
with basin_file.open('r') as fp:
    basins = sorted(basin.strip() for basin in fp if basin.strip())

for seed in seeds:
    kf = KFold(n_splits=n_kfold, random_state=None, shuffle=True)
    for kfold, (train_index, test_index) in enumerate(kf.split(basins)):
        train_basin_file = Path(f"/home/gsnearing/projects/data_assimilation/run_configs/pub/basin_files/train_{kfold}_{seed}.txt")
        test_basin_file = Path(f"/home/gsnearing/projects/data_assimilation/run_configs/pub/basin_files/test_{kfold}_{seed}.txt")
        with train_basin_file.open('w') as fp:
            for idx in train_index:
                fp.write(f"{basins[idx]}\n")
        with test_basin_file.open('w') as fp:
            for idx in test_index:
                fp.write(f"{basins[idx]}\n")   

In [25]:
master_config = "/home/gsnearing/projects/data_assimilation/master_configs/simulation_config.yml"
for seed in seeds:
    for kfold in range(n_kfold):
        with open(master_config, 'r') as f:
            filedata = f.read()
        filedata = filedata.replace("seed:", f"seed: {seed}")
        filedata = filedata.replace("experiment_name:", f"experiment_name: pub_simulation_{kfold}_{seed}")

        train_basin_file = f"/home/gsnearing/projects/data_assimilation/run_configs/pub/basin_files/train_{kfold}_{seed}.txt"
        test_basin_file = f"/home/gsnearing/projects/data_assimilation/run_configs/pub/basin_files/test_{kfold}_{seed}.txt"
        filedata = filedata.replace("train_basin_file: /home/gsnearing/projects/data_assimilation/531_basin_list.txt",
                                   f"train_basin_file: {train_basin_file}")
        filedata = filedata.replace("validation_basin_file: /home/gsnearing/projects/data_assimilation/531_basin_list.txt",
                                   f"validation_basin_file: {train_basin_file}")
        filedata = filedata.replace("test_basin_file: /home/gsnearing/projects/data_assimilation/531_basin_list.txt",
                                   f"test_basin_file: {test_basin_file}")

        new_config = f"/home/gsnearing/projects/data_assimilation/run_configs/pub/simulation/pub_simulation_{kfold}_{seed}.yml"
        with open(new_config, 'w') as f:
            f.write(filedata)

In [28]:
# PUB
master_config = "/home/gsnearing/projects/data_assimilation/master_configs/autoregression_holdout_config.yml"
for seed in seeds:
    for kfold in range(n_kfold):
        for lead in [1]:
            for holdout in [0.5]:
                
                with open(master_config, 'r') as f:
                    filedata = f.read()
                filedata = filedata.replace("seed:", f"seed: {seed}")
                filedata = filedata.replace("experiment_name:", f"experiment_name: pub_autoregression_{lead}_{holdout}_{kfold}_{seed}")
                filedata = filedata.replace(f"- xxx1\n", f"- {lead}\n")
                filedata = filedata.replace(f"- xxxQObs(mm/d)_shift1\n", f"- QObs(mm/d)_shift{lead}\n")
                filedata = filedata + f'random_holdout: \n '
                filedata = filedata + f'  QObs(mm/d)_shift{lead}: {holdout} \n '

                train_basin_file = f"/home/gsnearing/projects/data_assimilation/run_configs/pub/basin_files/train_{kfold}_{seed}.txt"
                test_basin_file = f"/home/gsnearing/projects/data_assimilation/run_configs/pub/basin_files/test_{kfold}_{seed}.txt"
                filedata = filedata.replace("train_basin_file: /home/gsnearing/projects/data_assimilation/531_basin_list.txt",
                                           f"train_basin_file: {train_basin_file}")
                filedata = filedata.replace("validation_basin_file: /home/gsnearing/projects/data_assimilation/531_basin_list.txt",
                                           f"validation_basin_file: {train_basin_file}")
                filedata = filedata.replace("test_basin_file: /home/gsnearing/projects/data_assimilation/531_basin_list.txt",
                                           f"test_basin_file: {test_basin_file}")

                new_config = f"/home/gsnearing/projects/data_assimilation/run_configs/pub/autoregression/pub_autoregression_{lead}_{holdout}_{kfold}_{seed}.yml"    
                with open(new_config, 'w') as f:
                    f.write(filedata)